In [1]:
import numpy as np
import xarray as xr
import glob

In [2]:
#for use on Casper
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(cores=36,
                     processes=36, memory="380GB",
                     project='P93300041',
                     walltime='2:00:00')
cluster.scale(36)
client = Client(cluster)

/glade/u/apps/dav/opt/python/3.7.5/gnu/8.3.0/pkg-library/20200417/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [46]:
#for use on cheyenne
from dask_jobqueue import PBSCluster
from dask.distributed import Client
cluster = PBSCluster(cores=32,
                     processes=32, memory="108GB",
                     project='P93300041',
                     queue='regular',
                     resource_spec='select=1:ncpus=36:mem=109G',
                     walltime='02:00:00')
cluster.scale(32)
client = Client(cluster)

/glade/u/apps/ch/opt/python/3.7.5/gnu/8.3.0/pkg-library/20200417/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [3]:
thedir = '/glade/scratch/charlie/archive/fates_clm50_global_4x5_historicaltransient_2e3f469f_2905a9ba/lnd/hist/'
paths = sorted(glob.glob(thedir+'*.h0.*'))

In [5]:
client

Client Scheduler: tcp://10.12.205.14:46100 Dashboard: http://10.12.205.14:36563/status,Cluster Workers: 36 Cores: 36 Memory: 380.16 GB


In [42]:
#client.close()

In [6]:
data_vars =['SECONDARY_FOREST_FRACTION',
            'PATCH_AREA_BY_AGE',
            'PFTcanopycrownarea',
            'SECONDARY_AREA_PATCH_AGE_DIST',
            'SECONDARY_AREA_AGE_ANTHRO_DIST',
            'NPLANT_SCLS',
            'ED_biomass',
            'area',
            'WOOD_PRODUCT',
            'GPP',
            'NPP']
def preprocess(ds):
    '''specify a smaller set of data_vars'''
    return ds[data_vars]

In [7]:
%%time 
ds=xr.open_mfdataset(paths,combine='by_coords',
                     preprocess=preprocess,parallel=True)

CPU times: user 30.5 s, sys: 2.28 s, total: 32.7 s
Wall time: 1min 1s


## If you want to add some more variables later

In [8]:
data_vars =['TWS']
def preprocess(ds):
    '''specify a smaller set of data_vars'''
    return ds[data_vars]

In [9]:
%%time 
ds2=xr.open_mfdataset(paths,combine='by_coords',
                      preprocess=preprocess,parallel=True)

CPU times: user 17.3 s, sys: 947 ms, total: 18.3 s
Wall time: 46.7 s


In [11]:
for v in data_vars:
    ds[v]=ds2[v]

In [12]:
ds

,Array,Chunk
Bytes,26.39 MB,13.25 kB
Shape,"(1992, 46, 72)","(1, 46, 72)"
Count,5976 Tasks,1992 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,184.73 MB,92.74 kB
Shape,"(1992, 7, 46, 72)","(1, 7, 46, 72)"
Count,5976 Tasks,1992 Chunks
Type,float32,numpy.ndarray
